## Pandas

In this notebook we will get to know the main features of **pandas**. This library is huge, so we can only play a little with the available functionality. For a detailed review, read the documentation.

### 1. First glance

In [ ]:
import pandas as pd  # common alias for pandas

In [ ]:
# Read dataset from csv file
coins = pd.read_csv('coins.csv')  

In [ ]:
# Let's see what we've got
coins

column descriptions:
 - date - date of measurement
 - name - coin's name
 - symbol - coin's symbol
 - price - average coin price per trading day in USD
 - txCount - the number of transactions in the coin network
 - txVolume - coin's volume transferred between the coin network addresses
 - activeAddresses - the number of addresses that transacted on the trading day in the coin network
 - open - coin price at the beginning of the trading day
 - close - coin price at the end of the trading day
 - high - highest coin price during the trading day
 - low - lowest coin price during the trading day
 - volume - trading volume of this coin on exchanges on the trading day
 - market - capitalization of this coin on the trading day

From output you can get a general idea of the table. You can see that the table can store both numeric and categorical (string) data, the number of rows and columns. All this information and much more can be obtained in the code.

For convenience, we will further limit the size of pandas output in jupyter notebook.

In [ ]:
pd.set_option('display.max_rows', 10)

In [ ]:
coins.shape  # Table's size

In [ ]:
coins.head(3)  #  First three rows of the table. use .tail(n) for a number of last rows

In [ ]:
coins.describe()  # Simple statistics for numeric columns

In [ ]:
coins['price']  # A column can be selected 

In [ ]:
coins['high'] - coins['low']  # element-wise operations work on columns

In [ ]:
coins['spread'] = coins['high'] - coins['low']  # New columns can be added to the table

In [ ]:
del coins['spread']  # Removing a column. Alternative: coins.pop('spread')

Columns in pandas tables behave like arrays of fixed length. The nice difference is that the columns have many useful analytic methods. For example,

In [ ]:
coins['symbol'].value_counts() # Number of rows for each coin

In [ ]:
# Maximum value for price column
coins['price'].max()

**Task**: How many different coins are in the dataset?

In [ ]:
# your code here

In [ ]:
# Standard indexing works
coins['txVolume'][:4]

Boolean masks also work

In [ ]:
coins['txCount'] > 1000

In [ ]:
# Rows with txCount > 1000
coins[coins['txCount'] > 1000]

In [ ]:
# coin's symbol is BTC or ADA
coins['symbol'].isin(['BTC', 'ADA'])

In [ ]:
# symbol contains 'C'
coins['symbol'].str.contains('C')

In [ ]:
# Complex conditions (brackets are mandatory!)
((coins['price'] > 18000) | (coins['market'] > 200000)) & (coins['symbol'] == 'BTC')

The main purpose of the resulting boolean columns is that they can be used as an indexing object. A kind of advanced slicing for columns and tables

In [ ]:
# Rows with BTC priced less than 100 USD
coins[(coins['price'] < 100) & (coins['symbol'] == 'BTC')]

In [ ]:
# Transactions count for litecoin
coins['txCount'][coins['symbol'] == 'LTC']

**Task**: What coin has the maximum price?

In [ ]:
# your code here

Table indexing works both for rows and columns

In [ ]:
# highest price and volume for days when DOGE has been priced greater than 1 cent
coins.loc[
    (coins['symbol'] == 'DOGE') & (coins['price'] > 0.01),
    ['high', 'volume']
]

In [ ]:
# TAB-completions!!!
list.

From now on, you can already make simple sets from the data and transform them a bit. Of course, remember to use help() and tab-completion to learn more about the features of pandas. It’s worth keeping this rule in mind: if you really want to use a loop to iterate through the table and do something with the data, it is very likely that you are doing something wrong and, most likely, pandas has a tool for your task. The rest of this notebook is dedicated to pandas features, and it may seem boring, but without knowing these features you will be reinventing the wheel each time.

### 2. Global settings

You might want to configure pandas. Most global settings relate to displaying a table or data.

In [ ]:
# get the maximum number of columns being displayed
pd.get_option('display.max_columns')

In [ ]:
# To find out more about an option.  
# The full list of options: https://pandas.pydata.org/pandas-docs/version/0.23.4/options.html#available-options
pd.describe_option('display.max_columns')

In [ ]:
# set the maximum number of columns being displayed
pd.set_option('display.max_columns', 15)

### 3. Data types in pandas

There are two main types of data in pandas - pandas.Series and pandas.DataFrame.

#### 3.1 pandas.Series

Pandas.Series should be thought of as a one-dimensional array of a fixed size with data of the same type. Unlike arrays, access to elements can be at a non-numeric index. An index should be understood as row name.

pandas.Series objects are very often used as return values in pandas.

In [ ]:
# pandas.Series with standard indexes
pd.Series(data=['BTC', 'LTC', 'DOGE', 'DASH'])

In [ ]:
# pandas.Series with str indexes
pd.Series(data=['BTC', 'ETH', 'XRP'], 
          index=['first_coin', 'second_coin', 'third_coin'], dtype=str)

In [ ]:
# casting a dictionary
pd.Series({'first_coin': 'BTC', 'second_coin': 'ETH', 'third_coin': 'XRP'})

In [ ]:
sequence = pd.Series(data=['!', 'the', 'Moon', 'To'])
sequence[[3, 1, 2, 0, 0, 0]]  # Access elements by indexes

In [ ]:
sequence[3]  # Getting an element by its index

In [ ]:
# Works with non-standard indexes too
sequence = pd.Series(data=['!', 'the', 'Moon', 'To'], 
                     index=['04-11-2018', '02-11-2018', '03-11-2018', '01-11-2018'])

In [ ]:
sequence[['01-11-2018', '02-11-2018', '03-11-2018', '04-11-2018']] 

In [ ]:
sequence['01-11-2018'] 

In [ ]:
# Remove an object
del sequence

#### 3.2 pandas.DataFrame

pandas.DataFrame is a two-dimensional array of variable size with heterogeneous data (but of the same type within each column). It’s better to think of pandas.DataFrame as a set of columns, where each column is of pandas.Series type.

In [ ]:
# Create a table
pd.DataFrame(data=['BTC', 'LTC', 'XRP'])

In [ ]:
# Create a table with specified names of columns  
pd.DataFrame(data=[['BTC', 10000],['LTC', 200],['XRP', 1]], 
             columns=['symbol', 'price'])

In [ ]:
# And now with specific indexes also
pd.DataFrame(
    data=[['BTC', 10000],['LTC', 200],['XRP', 1]],
    columns=['symbol', 'price'],
    index=pd.DatetimeIndex(['01-11-2018', '03-11-2018', '23-08-2018'])
)

In [ ]:
# Create a table from a set of pandas.Series
pd.DataFrame({
    'symbol': pd.Series(data=['BTC','LTC'], index=pd.DatetimeIndex(['01-11-2018', '03-11-2018'])),
    'price': pd.Series(data=[10000, 200], index=pd.DatetimeIndex(['01-11-2018', '03-11-2018']))
})

### 4. Basic methods of pandas.Series

In [ ]:
sequence = pd.Series(data=['BTC', 'LTC', 'DOGE', 'DASH'], index=['leader', 'alternative', 'joke', 'bouncer'])

In [ ]:
# Get indexes
sequence.axes

In [ ]:
# Get type of the data
sequence.dtype

In [ ]:
# Check if empty
sequence.empty

In [ ]:
# Number of dimensions. Always 1 for pandas.Series
sequence.ndim

In [ ]:
# Number of elements
sequence.size

In [ ]:
# Sizes
sequence.shape

In [ ]:
# Cast to numpy array
sequence.values

In [ ]:
# Get first three elements
sequence.head(3)

In [ ]:
# Get last two elements
sequence.tail(2)

In [ ]:
del sequence

### 5. Basic methods of pandas.DataFrame

DataFrame is huge so we will look only at the most frequently used methods


In [ ]:
table = pd.DataFrame(
    data=[['BTC', 10000],['LTC', 200],['XRP', 1]],
    columns=['symbol', 'price'],
    index=pd.DatetimeIndex(['01-11-2018', '03-11-2018', '23-08-2018'])
)

In [ ]:
# Transpose table
table.T

In [ ]:
# Get indexes
table.axes

In [ ]:
# Get data types
table.dtypes

In [ ]:
# Check if empty
table.empty

In [ ]:
# Sizes
table.shape

In [ ]:
# Number of cells
table.size

In [ ]:
# Cast to numpy ndarray
table.values

In [ ]:
# Get first three rows
table.head(3)

In [ ]:
# Get last two rows
table.tail(2)

In [ ]:
del table

### 6. Working with rows and columns

We can get rows either by indexes or by ordinal numbers

In [ ]:
table = pd.DataFrame(
    data=[['BTC', 10000],['LTC', 200],['XRP', 1], ['BTC', 9000]],
    columns=['symbol', 'price'],
    index=pd.DatetimeIndex(['01-11-2018', '03-11-2018', '23-08-2018', '02-11-2018'])
)
table

#### 6.1. Getting rows

In [ ]:
# By ordinal number
table.iloc[2]

In [ ]:
# By range of numbers
table[1:3]

In [ ]:
# By index
table.loc[pd.Timestamp('01-11-2018')]

In [ ]:
# By range of indexes
table.loc['01-11-2018':'03-11-2019']

#### 6.2. Adding, removing and renaming rows and columns

In [ ]:
# Remove rows by indexes.  Can be used for column removal with axis=1 parameter.
# See also drop_duplicates and dropna
table.drop([pd.Timestamp('2018-08-23'), pd.Timestamp('2018-01-11')])

In [ ]:
# Add new column
table['volume'] = [1e9, 1e6, 1e4, 1e9]
table

In [ ]:
# Remove a column
del table['volume']

In [ ]:
# Rename columns
table.rename(columns={'symbol': 'Symbol', 'price': 'Price'})

#### 6.3. Iterating

Iterating through a table can be done in a number of ways. But better not to use loops at all, use pandas methods instead. 

In [ ]:
# By columns
for column in table:
    print(column)

In [ ]:
# By columns as dictionaries
for column, series in table.iteritems():
    print(column)
    print(series)

In [ ]:
# By rows as dictionaries
for index, row in table.iterrows():
    print(index)
    print(row)

#### 6.4. Sorting

Rows and columns of the table can be sorted. Sorting methods return a new table!

In [ ]:
# Sort rows in decreasing order of index
table.sort_index(ascending=False)

In [ ]:
# Sort columns in increasing order of names
table.sort_index(axis=1, ascending=True)

In [ ]:
# Sort rows by symbol, price using quicksort algorithm
table.sort_values(by=['symbol', 'price'], kind='quicksort')

### 7. Stat functions

Let's see how pandas allows you to calculate the simplest statistics for datasets. The methods that we list below have the axis argument, which indicates along which axis statistics should be calculated. By default axis = 0, which corresponds to row-based calculations. If there are gaps in the data, then they are not taken into account when calculating statistics!

In [ ]:
earnings = pd.DataFrame(
    data=[[7629.39, -9357.49, -1661.3, 8597.23],
          [560.68, None, 10.46, 3578.5],
          [487.38, 7560.38, 1090.87, -5164.93]],
    columns=['BTC', 'DOGE', 'ADA', 'ETH'],
    index=['yesterday', 'today', 'tomorrow']
)
earnings

In [ ]:
# Count non-NaN values for each column
earnings.count()

In [ ]:
# earnings by day
earnings.sum(axis=1)

In [ ]:
# Mean values
earnings.mean()

In [ ]:
# Median values
earnings.median()

In [ ]:
# Modes
earnings.mode()

In [ ]:
# Standard deviations
earnings.std()

In [ ]:
# Quantiles
earnings.quantile(q=(0.25, 0.5, 0.75))

In [ ]:
# Minimum values
earnings.min()

In [ ]:
# Maximum values
earnings.max()

In [ ]:
# Absolute values
earnings.abs()

In [ ]:
# Productions of all values within each column. I know it makes no sense here
earnings.prod()

In [ ]:
# Cumulative productions
earnings.cumprod()

In [ ]:
# Cumulative sums
earnings.cumsum()

In [ ]:
earnings

In [ ]:
# Percent changes comparing with previous row
earnings.pct_change()

In [ ]:
# Covariance between BTC and DOGE
earnings['BTC'].cov(earnings['DOGE'])

In [ ]:
# Correlation between BTC and DOGE
earnings['BTC'].corr(earnings['DOGE'])

One describe() to rule them all

In [ ]:
earnings.describe()

In [ ]:
del earnings

### 8. Applying custom functions

Sometimes you want to apply a function to a dataset or one of its columns. You can apply them to the entire table, element-wise, column-wise or row-wise

In [ ]:
earnings = pd.DataFrame(
    data=[[7629.39, -9357.49, -1661.3, 8597.23],
          [560.68, None, 10.46, 3578.5],
          [487.38, 7560.38, 1090.87, -5164.93]],
    columns=['BTC', 'DOGE', 'ADA', 'ETH'],
    index=['yesterday', 'today', 'tomorrow']
)

To apply in row-based way, use the apply function. If you want column-based way add axis = 1

In [ ]:
# Calculate dispersions. Lambda takes pandas.Series of each column
earnings.apply(lambda money: money.std() ** 2)

To apply a function element by element use applymap

In [ ]:
# Let's pretend that the losses turned into a profit
earnings.applymap(abs)

In [ ]:
# Apply to one column
earnings['BTC'].apply(lambda money: money + 100500)

In [ ]:
earnings.pipe() # Figure it out by yourself

In [ ]:
del earnings

### 9. Working with string columns

At the moment, to work with string columns, we would have to constantly use the apply method. This is inconvenient and therefore pandas has convenient string manipulation tools

In [ ]:
taxonomy = pd.DataFrame(
    data=[['BTC', 'Bitcoin'],['LTC', 'Litecoin'], ['ETC', 'Etherium'], ['DOGE', 'Doge coin']],
    columns=['symbol', 'name']
)
taxonomy

To apply string functions to a column, you need to refer to the .str property.

In [ ]:
# To lowercase
taxonomy['symbol'].str.lower()

In [ ]:
# To uppercase
taxonomy['name'].str.upper()

In [ ]:
# String lengths
taxonomy['symbol'].str.len()

In [ ]:
# Strip strings
taxonomy['name'].str.strip()

In [ ]:
# Split strings
taxonomy['name'].str.split(' ')

In [ ]:
# Concatenate (analogous to str.join)
taxonomy['symbol'].str.cat(sep=' $$$ ')

In [ ]:
# Find substring
taxonomy['name'].str.contains('coin')

In [ ]:
# Replace substring by another substring
taxonomy['name'].str.replace('coin', 'dough')

In [ ]:
# Repeat string N times
taxonomy['name'].str.repeat(2)

In [ ]:
# Count substring occurences
taxonomy['name'].str.count('coin')

In [ ]:
# Check prefix
taxonomy['symbol'].str.startswith('B')

In [ ]:
# Check suffix
taxonomy['symbol'].str.endswith('C')

In [ ]:
# Index of substring leftmost occurence
taxonomy['name'].str.find('coin')

In [ ]:
# Check if all in uppercase
taxonomy['symbol'].str.isupper()

In [ ]:
# Check if all in lowercase
taxonomy['symbol'].str.islower()

In [ ]:
# Check if numeric
taxonomy['name'].str.isnumeric()

### 10. Aggregations, transformations, and filtration

#### 10.1. Rolling windows

In [ ]:
price = pd.DataFrame(
    data=[
        [0.1, 0.2],
        [0.2, 0.3],
        [0.3, 0.4],
        [0.4, 0.5],
        [0.5, None],
        [0.6, 0.7]],
    columns=['ADA', 'DOGE'],
    # Date range of 6 days starting at 2018-11-01 
    index=pd.date_range('2018-11-01', periods=6)
)

In [ ]:
price

If you want to calculate some statistical function (sum, mean, median, std) in a sliding window you can use rolling method (normal window) and ewm (window with exponential weights). They return special objects that can aggregate using a standard or custom function.

In [ ]:
# Rolling mean
price.rolling(window=2).mean()

In [ ]:
#Expanding mean
price.expanding().mean()

In [ ]:
# And now with custom function
price.rolling(window=2).agg(lambda series: sum(series ** 2))

In [ ]:
# Aggregation by function's name
price.rolling(window=2).agg('max')

In [ ]:
# Several aggregations
price.rolling(window=2).agg(['mean', sum, lambda series: max(series ** 2)])

In [ ]:
# Several aggregations different for each column
price.rolling(window=2).agg({'ADA': [sum, max], 'DOGE': lambda series: max(series ** 2)})

The ewm object works similarly to rolling, with the only difference being that it assigns weights. Values $x_0,\ldots,x_t$ will be assigned with weights $(1-\alpha)^t, (1-\alpha)^{t-1}, \ldots, 1$, where $\alpha$ - smoothing parameter. For example, exponential weighted moving average will be calculated like this:

$$
ewm_{\alpha}(x)=\frac{\sum_{i=0}^t (1-\alpha)^{t-i} x_i}{\sum_{i=0}^t(1-\alpha)^i}
$$

To use simpler form $y_i = \alpha*x_i + (1-\alpha)*y_{i-1}$ specify adjust=False

In [ ]:
# EWM mean
price.ewm(alpha=0.5, adjust=False).mean()

In [ ]:
del price

#### 10.2. Grouping

Sometimes you want to break the data down into groups and calculate statistics or filter. There is a groupby method for this. It returns a special object for working with grouped data.

In [ ]:
operations = pd.DataFrame(
    data=[
        ['DOGE', 'buy', 5000, 0.2],
        ['BTC', 'buy', 10, 6000],
        ['BTC', 'buy', 2.5, 5900],
        ['DOGE', 'sell', 3000, 0.1],
        ['BTC', 'sell', 4, 6200],
        ['ETH', 'sell', 10, 400],
        ['BTC', 'buy', 1, 5600],
        ['ETH', 'buy', 20, 350],
        ['ETH', 'sell', 10, 300],
        ['DOGE', 'sell', 3000, 0.15]
    ],
    columns=['symbol', 'type', 'amount', 'price'],
    index=pd.date_range('2018-01-01', periods=10)
)

In [ ]:
operations

In [ ]:
# Group by symbol
operations.groupby('symbol')

In [ ]:
# Group by symbol and type
operations.groupby(['symbol', 'type'])

In [ ]:
# Get indexes for each group
operations.groupby(['symbol', 'type']).groups

In [ ]:
# get each group
for name, group in operations.groupby(['symbol', 'type']):
    print(name)
    print(group)

In [ ]:
# get a group
operations.groupby(['symbol', 'type']).get_group(('BTC', 'buy'))

And now we will aggregate

In [ ]:
# Get buy and sell volumes for each coin
grouper = operations.groupby(['symbol', 'type'])
grouper['amount'].agg(sum)

In [ ]:
# Get buy and sell volumes and biggest deals for each coin
grouper = operations.groupby(['symbol', 'type'])
grouper['amount'].agg([sum, max])

In [ ]:
# Get buy and sell volumes and mean price for each coin
grouper = operations.groupby(['symbol', 'type'])
grouper.agg({'amount': sum, 'price': lambda series: series.mean()})

Grouped data can not only be aggregated but also transformed.

In [ ]:
# Get group sizes
operations.groupby(['symbol', 'type']).transform(lambda series: series.count())

Grouped data can be filtered

In [ ]:
# Filter by the volume > 20
operations.groupby(['symbol', 'type']).filter(lambda table: table['amount'].sum() > 20)

In [ ]:
del operations, name, group

#### 10.3 Pivot tables

Most raw tables look like logs where each row is a record. There is no organization in the data in these tables. There are pivot tables in pandas for this.

In [ ]:
operations = pd.DataFrame(
    data=[
        ['DOGE', 'buy', 5000, 0.2, 'Joe Doe', 'success'],
        ['BTC', 'buy', 10, 6000, 'Elon Musk', 'error'],
        ['BTC', 'buy', 2.5, 5900, 'George Bush', 'success'],
        ['DOGE', 'sell', 3000, 0.1, 'John Romero', 'success'],
        ['BTC', 'sell', 4, 6200, 'Jack Ma', 'success'],
        ['ETH', 'sell', 10, 400, 'Satoshi Nakomoto', 'error'],
        ['BTC', 'buy', 1, 5600, 'Vladimir Vladimirovich', 'success'],
        ['ETH', 'buy', 20, 350, 'George Bush', 'success'],
        ['ETH', 'sell', 10, 300, 'Jack Ma', 'error'],
        ['DOGE', 'sell', 3000, 0.15, 'Joe Doe', 'success']
    ],
    columns=['symbol', 'type', 'amount', 'price', 'user', 'status'],
    index=pd.date_range('2018-01-01', periods=10)
)

In [ ]:
operations

The best way to understand how they work is to parse an example. We will find how much in total each type of coin was bought and sold. In the resulting summary table, the index will be the name of the coin, the columns are the types of transactions (i.e., purchases or sales). In the cells themselves there will be a total amount of coins.

In [ ]:
operations.pivot_table(
    values='amount', 
    index='symbol', 
    columns='type', 
    aggfunc='sum'
)

By default aggfunc is avg. Let's consider more complex example:

In [ ]:
operations.pivot_table(
    values='amount',
    index=['symbol', 'user'],  # Index can be multi-columned
    columns=['type', 'status'],  # So as columns
    aggfunc={'amount': ['sum', 'mean']}  # Several aggregations can be calculated
)

And the most monstruose example:

In [ ]:
operations.pivot_table(
    values=['amount', 'price'],  # Several values
    index=['symbol', 'user'],
    columns=['type', 'status'],
    aggfunc={
        'amount': ['sum', 'mean'], 
        'price': 'max'}  # with specific aggfuncs
)

### 11. Gaps, errors and NaNs

In [ ]:
price = pd.DataFrame(
    data=[
        [0.547, 0.745],
        [-2, -1],
        [None, 0.718],
        [0.135, 0.845],
        [0.53, None],
        [None, -1]],
    columns=['ADA', 'DOGE'],

    index=pd.date_range('2018-11-01', periods=6)
)

In [ ]:
price

In [ ]:
# Find all NaNs. There is also notnull()
price.isnull()

In [ ]:
# Replace NaNs with specified default value
price.fillna(0)

In [ ]:
# Replace NaNs with next values (backward fill)
price.fillna(method='bfill')

In [ ]:
# Replace NaNs with previous values (forward fill)
price.fillna(method='ffill')

In [ ]:
# Drop rows with NaNs
price.dropna()

In [ ]:
# Drop columns with NaNs
price.dropna(axis=1)

In [ ]:
# Replace values
price.replace({-1:0, -2:0})

In [ ]:
del price

### 12. Merging and concatenating

If you have two tables, then the information in them can be combined in two ways: concatenation ("stick a new table from the bottom") and join (find the union of table entries if they have the same values in the selected columns).

In [ ]:
operations_old = pd.DataFrame(
    data=[
        ['DOGE', 'buy', 5000, 0.2],
        ['BTC', 'buy', 10, 6000],
        ['BTC', 'buy', 2.5, 5900],
        ['DOGE', 'sell', 3000, 0.1],
        ['BTC', 'sell', 4, 6200],
    ],
    columns=['symbol', 'type', 'amount', 'price']
)

operations_new = pd.DataFrame(
    data=[
        ['ETH', 'sell', 10, 400],
        ['BTC', 'buy', 1, 5600],
        ['ETH', 'buy', 20, 350],
        ['ETH', 'sell', 10, 300],
        ['DOGE', 'sell', 3000, 0.15]
    ],
    columns=['symbol', 'type', 'amount', 'price']
)
print(operations_old)
print(operations_new)

In [ ]:
#  Concatenate in a specified order
pd.concat([operations_old, operations_new])

In [ ]:
# You can use tags to remember to which table a row belongs
pd.concat([operations_old, operations_new], keys=['old', 'new'])

In [ ]:
# to update indexes use ignore_index=True
operations = pd.concat([operations_new, operations_old], ignore_index=True)
operations

Data can be collected together in a smarter way. Namely, you choose which columns you are interested in. Further, if there are records from two tables which have the same values in the selected columns, we "combine" them. This is a very simplified explanation. In pandas, this operation is called merge (in SQL it is join). You can do the union in different ways, so several strategies arise: left, right, inner and full.

In [ ]:
transactions = pd.DataFrame(
    data=[
        ['DOGE', 'sell', 'Rockfeller'],
        ['DOGE', 'buy', 'J.P. Morgan'],
        ['BTC', 'buy', 'John Doe'],
        ['ADA', 'sell', 'Rick'],
        ['ETH', 'buy', 'Morty']
    ],
    columns=['symbol', 'type', 'user']
)
transactions

In [ ]:
# Merge by symbol and type
# By default it's inner join
pd.merge(operations, transactions, on=['symbol', 'type'])

In [ ]:
# Left join
# (right join exists also)
pd.merge(operations, transactions, on=['symbol', 'type'], how='left')

In [ ]:
# Outer join
pd.merge(operations, transactions, on=['symbol', 'type'], how='outer')

In [ ]:
del operations_new, operations_old, operations, transactions

### 13. Read table

In order to analyze data you should get it somewhere. In pandas there are functions with *read_* prefix for that. We will look at read_csv.

In [ ]:
# Just read a table from file
pd.read_csv('coins.csv')

In [ ]:
# Read and use date column as index
pd.read_csv('coins.csv', index_col='date')

In [ ]:
# Read and help pandas with data types for two columns
pd.read_csv('coins.csv', dtype={'symbol': str, 'name': str})

### 14. Visualization

Pandas has very simple visualization tools. They are suitable for quick analysis, but if you want to do it beautifully and informatively, then you are not at the right place. Any visualization is done through the plot attribute of the pandas.Series or pandas.DataFrame object.

Remember about this line in order to see your plots:
```
%matplotlib inline
```

In [ ]:
%matplotlib inline

In [ ]:
price = pd.DataFrame(
    data=[
        [0.547, 0.745],
        [0.5, 0.964],
        [0.77, 0.718],
        [0.135, 0.845],
        [0.53, None],
        [0.15, 0.795]],
    columns=['ADA', 'DOGE'],
    index=pd.date_range('2018-11-01', periods=6)
)
price

In [ ]:
# Plot prices for ADA coin
price['ADA'].plot()

In [ ]:
# Plot several price graphs
price[['ADA', 'DOGE']].plot()

In [ ]:
# Bar chart
price.plot.bar(stacked=True)

In [ ]:
# Horizontal bar chart
price.plot.barh()

In [ ]:
# Histogram
price.plot.hist(bins=10)

In [ ]:
# Box plot
price.plot.box()

In [ ]:
# Filled plots
price.plot.area()

In [ ]:
# Scatter plot
price.plot.scatter(x='ADA', y='DOGE')

In [ ]:
# Pie chart
price.plot.pie(subplots=True, figsize=(15,7.5))

In [ ]:
del price

# NB


## Other tools to work with data

If your dataset is not too big, then pandas is de facto the best tool available. Otherwise, there are alternatives:
 - [numpy](http://www.numpy.org/) - library for low-level work with multidimensional arrays of primitive data types. Suitable for "vectorized" processing. Numpy is your last chance to still write in regular python without C bindings. Suitable for tasks of fast processing of not too large data. Actually pandas uses numpy under its hood.
 - [Graphlab](https://turi.com/) - Paid Python library for working with both big data and small datasets. Data is represented as SFrame objects, much like the DataFrame from pandas.
 - [Hadoop](https://hadoop.apache.org/) - for working with very large data, in particular there is a classic MapReduce.
 
## Other visualization tools


There are many more libraries for visualization. Here are the most popular
 - [Pandas](https://pandas.pydata.org/) - for quick and very easy visualization. Any customization will require knowledge of matplotlib
 - [Seaborn](https://seaborn.pydata.org/) - A wide but fixed set of sophisticated visualizations with beautiful color schemes. The library is an add-on for matplotlib, and again, fine tuning will require knowledge of matplotlib.
   - [Gallery with examples](https://seaborn.pydata.org/examples/index.html)
   - [Tutorial](https://seaborn.pydata.org/tutorial.html)
 - [ggplot](http://ggplot.yhathq.com/) - promising library copied from R. So far at the development stage
 - [Bokeh](https://bokeh.pydata.org/en/latest/) - for visualization in web
 - [Plot.ly](https://plot.ly/) - library for interactive and static visualizations. It is aimed at working in the cloud, but it can work locally.
   - [Gallery with examples](https://plot.ly/python/)
   - [Cheat sheet](https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf)